In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.svm import SVC, LinearSVC
#from sklearn.linear_model import SGDClassifier
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
#from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.ensemble import HistGradientBoostingClassifier
from joblib import dump, load
#from sklearn.naive_bayes import GaussianNB 
from VisionUtils import *
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from mtcnn.mtcnn import MTCNN
from pathlib import Path
from sklearn.utils import shuffle
from tqdm import tqdm
#import autosklearn.classification

##### !curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

In [2]:
def trans(feat1, feat2):
    f1 = [feat1[i1].reshape(-1,1) for i1 in range(len(feat1))]
    f2 = [feat2[i2].reshape(-1,1) for i2 in range(len(feat2))]
    cos_d = np.array([feat_distance_cosine_scalar(f1[i].T, f2[i]) for i in range(len(feat1))])
    cos_d = cos_d.reshape(-1,1)
    sqr_diff = np.power(np.abs(feat1- feat2), 2)
    rat = feat1/feat2
    data = np.hstack([cos_d, sqr_diff, rat])
    return data

train_aug_feat1 = np.load("features/train_aug_feat12.npy")
train_aug_feat2 = np.load("features/train_aug_feat22.npy")
train_aug_labels = np.load("features/train_aug_labels2.npy")

train_parallel_feat1 = np.load("features/train_parallel_feat11.npy")
train_parallel_feat2 = np.load("features/train_parallel_feat21.npy")
train_parallel_labels = np.load("features/train_parallel_labels.npy")

train_spotlight_feat1 = np.load("features/train_spotlight_feat11.npy")
train_spotlight_feat2 = np.load("features/train_spotlight_feat21.npy")
train_spotlight_labels = np.load("features/train_spotlight_labels.npy")

In [3]:
train_feat1_o = np.load("features/train_feat1.npy")
train_feat2_o = np.load("features/train_feat2.npy")
train_labels_o = np.load("features/train_labels.npy")

train_feat1_aug = np.load("features3/train_feat1.npy")
train_feat2_aug = np.load("features3/train_feat2.npy")
train_labels_aug = np.load("features3/train_labels.npy")

train_feat1_mb = np.load("features_motion_blur/train_feat1.npy")
train_feat2_mb = np.load("features_motion_blur/train_feat2.npy")
train_labels_mb = np.load("features_motion_blur/train_labels.npy")

train_feat1_som = np.load("features2/train_feat1.npy")
train_feat2_som = np.load("features2/train_feat2.npy")
train_labels_som = np.load("features2/train_labels.npy")

#test_feat1 = np.load("features/test_feat1.npy")
#test_feat2 = np.load("features/test_feat2.npy")
#test_labels = np.load("features/test_labels.npy")

#test_feat1 = np.load("features2/test_feat1.npy")
#test_feat2 = np.load("features2/test_feat2.npy")
#test_labels = np.load("features2/test_labels.npy")

test_feat1 = np.load("features/test_feat1.npy")
test_feat2 = np.load("features/test_feat2.npy")
test_labels = np.load("features/test_labels.npy")

In [5]:
labels = list(train_labels_o) + list(train_labels_aug) + list(train_labels_mb)+ list(train_labels_som) + list(test_labels) 

In [6]:
len(labels)

651092

In [7]:
len(labels)

651092

In [8]:
train_feat1 = np.vstack([train_feat1_o, train_feat1_aug, train_feat1_mb, train_feat1_som])
train_feat2 = np.vstack([train_feat2_o, train_feat2_aug, train_feat2_mb, train_feat2_som])


In [9]:
train_feat1.shape

(634937, 128)

In [10]:
test_feat1.shape

(16155, 128)

In [11]:
test_feat2.shape

(16155, 128)

In [12]:
xtrain = trans(train_feat1, train_feat2)
xtest = trans(test_feat1, test_feat2)

In [13]:
tr_len = len(xtrain)
tst_len = len(xtest)

In [14]:
data = np.vstack([xtrain, xtest])


In [15]:
train_feat1

array([[ 0.04792953,  0.7563431 , -0.84855455, ..., -0.22007652,
        -0.274067  ,  0.25541857],
       [ 2.084942  , -0.3050092 , -1.309191  , ...,  0.5104576 ,
         1.4823234 ,  1.0534424 ],
       [-0.28587088, -0.43783346, -0.6955893 , ...,  0.9054784 ,
         0.47151074, -0.8195881 ],
       ...,
       [-0.14162165,  0.566401  , -0.29851282, ..., -0.9206232 ,
         0.07092834,  0.22003533],
       [ 1.8893486 ,  0.30587184,  0.5176531 , ...,  0.7488258 ,
         0.22750878, -1.5933679 ],
       [-0.0756229 , -0.5898951 , -0.41407582, ..., -0.9840678 ,
         0.84356314,  0.14764242]], dtype=float32)

In [16]:
train_feat1.shape

(634937, 128)

In [17]:
train_feat2.shape

(634937, 128)

In [18]:
test_feat1.shape

(16155, 128)

In [19]:
test_feat2.shape

(16155, 128)

In [20]:
data.shape
#labels.shape

(651092, 257)

In [21]:
len(labels)

651092

In [22]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
f_selector = SelectKBest(chi2, k = int(0.8*(len(data.T))))
data_ = f_selector.fit_transform(scaled_data, labels )

data_df = pd.DataFrame(data_)
image_paths_csv = pd.read_csv("features/image_paths.csv", index_col = False)

all_data_df = pd.concat([image_paths_csv,data_df], axis = 1)

len(all_data_df)

xtrain, xtest, ytrain, ytest = train_test_split(all_data_df, labels, shuffle = True, random_state = 42)

paths_dict = {
    "train_paths1": xtrain["path1"],
    "train_paths2": xtrain["path2"],
    "test_paths1": xtest["path1"],
    "test_paths2": xtest["path2"]
}
xtrain1 = xtrain.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values
xtest1 = xtest.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values

In [23]:
xtrain, xtest = data_[:tr_len], data_[-tst_len:]
ytrain, ytest = labels[:tr_len], labels[-tst_len:]
xtrain, ytrain = shuffle(xtrain, ytrain)

In [24]:
xtrain.shape

(634937, 205)

In [25]:
xtest.shape

(16155, 205)

Gradient Boosting Classifier

In [26]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


params_grid = {
    "loss" : ["exponential"],
    "n_estimators": [60,70,90], "max_depth": 10
              }
gboost = GradientBoostingClassifier()
gs = GridSearchCV(gboost, param_grid = params_grid, cv = 2 )
gs.fit(xtrain, ytrain)
report(gs.cv_results_)
gboost = gs.best_estimator_

In [27]:
gboost = GradientBoostingClassifier(loss='exponential', n_estimators=60, max_depth=6)

params_grid = {
    "loss" : ["deviance", "exponential"],
    "n_estimators": [10, 50,100, 150, 200],}

gs = GridSearchCV(gboost, param_grid = params_grid, cv = 5 )

gboost.fit(xtrain, ytrain)

gboost = gs.best_estimator_

In [28]:
gboost.fit( xtrain, ytrain)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=60,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [29]:
pred = gboost.predict(xtest)

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.8886412875270814
-The recall Score:  0.855923818946327
-The precision score:  0.9160820648577102
-The F1_score:  0.8849817786586536
-The confusion matrix:
[[7435  634]
 [1165 6921]]


In [282]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.7332710280373832
-The recall Score:  0.7061419639573373
-The precision score:  0.7535321821036107
-The F1_score:  0.7290677805202203
-The confusion matrix:
[[2003  628]
 [ 799 1920]]


In [233]:
false_positive = []
false_positive_pred = []

false_negative = []
false_negative_pred = []

for i in range(len(pred)):
    ### False Positive check
    if ((pred[i]==1) & (ytest[i] ==0)):
        false_positive.append(i)
        false_positive_pred.append(pred[i])
    ### False Negative check
    elif ((pred[i] == 0) &(ytest[i]==1)):
        false_negative.append(i)
        false_negative_pred.append(pred[i])


In [234]:
print(len(false_positive))
print(len(false_negative))

7741
14150


False Positves

In [235]:
false_image_lst1 = list(np.array(paths_dict["test_paths1"])[false_positive])
false_image_lst2 = list(np.array(paths_dict["test_paths2"])[false_positive])

NameError: name 'paths_dict' is not defined

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

def showfalse(idx, i):
    
    try:
        imgs = [ plt.imread("data/images/" + false_image_lst1[idx]), 
               plt.imread("data/images/" + false_image_lst2[idx])]
        pathls = ["#: {} data/images/".format(i) + false_image_lst1[idx], 
                  "#: {} data/images/".format(i) + false_image_lst2[idx]]
        for i in range(len(imgs)):
            bb = detect_faces(imgs[i], m)
            ax = show_img(imgs[i])
            
            for b in bb:
                write_txt(ax, (b[0], b[1]),pathls[i], 14)
                draw_bb(ax, b)
            write_txt(ax, (0, 0),"{}".format(i), 20)
        #print(50*"#")
    except FileNotFoundError:
        print(i)

In [31]:
dump(gboost, "demo/PipelineParts/GboostModel.joblib")
dump(scaler, "demo/PipelineParts/scaler.joblib")
dump(f_selector, "demo/PipelineParts/feature_selector.joblib")

['demo/PipelineParts/feature_selector.joblib']

In [32]:
gboost

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=60,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [33]:
gboost = load("demo/PipelineParts/GboostModel.joblib")

In [ ]:
pred = gboost.predict(xtest)

In [207]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.8774155405405405
-The recall Score:  0.8422031691272217
-The precision score:  0.9057379285259616
-The F1_score:  0.8728158572705446
-The confusion matrix:
[[54084  5183]
 [ 9331 49802]]
